In [2]:
class Leaf:
    def __init__(self,left,right=''):
        if len(left) == 0:
            self.left = '$'
            self.right = 1
        else:
            self.left = left
            self.right = Leaf(right)
    
    def __repr__(self): return '(' + repr(self.left) + ',' + repr(self.right) + ')'

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.left == '$': return 0
        return len(self.left)+len(self.right)
    
    # def __str_(self): return '\t' + str(self.left) + ':' + str(self.right) + '\n'

In [3]:
class Branch:
    def __init__(self):
        self.b = {}

    def __repr__(self): 
        s = ''
        for a in self.b.values(): 
            s += repr(a) 
        return s

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index): return self.b[index]
    
    def __contains__(self,other): return other in self.b

    def add(self,leaf):
        if leaf in self.b:
            if self.b[leaf].left == '$':
                self.b[leaf].right+=1
                return
            i = 0
            left = ''
            while i < min(len(leaf.left),len(self.b[leaf].left)) and leaf.left[i] == self.b[leaf][i]: 
                left += leaf.left[i]
                i += 1
            if type(self.b[leaf].right) == Branch:
                nleaf = Leaf(leaf.left[i:])
                if len(self.b[leaf].left[i:]):
                    branch = Branch()
                    l = Leaf(self.b[leaf].left[i:])
                    l.right = self.b[leaf].right
                    branch.b[l] = l
                    self.b[leaf].right = branch
                self.b[leaf].right.add(nleaf)
            else: 
                branch = Branch()
                branch.add(Leaf(self.b[leaf].left[i:]))
                branch.add(Leaf(leaf.left[i:]))
                self.b[leaf].right = branch
            self.b[leaf].left = left
        else: 
            self.b[leaf] = leaf

In [11]:
b = Branch()
texts = ['hello','hollow','bellow']
for text in texts:
    for i in range(len(text)): b.add(Leaf(text[i:]))
print(b)

('h',('ello',('$',1))('ollow',('$',1)))
('ello',('$',1)('w',('$',1)))
('l',('lo',('$',1)('w',('$',2)))('o',('$',1)('w',('$',2))))
('o',('$',1)('llow',('$',1))('w',('$',2)))
('w',('$',2))
('bellow',('$',1))


In [ ]:
b.b

In [109]:
b = Branch()
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                                     'r_butter_to_make_the_',
                                                                            'r_to_make_the_bitt',
                                                                                   'ke_the_bitter_butter_better']
for read in reads:
    for i in range(len(read)):
        b.add(Leaf(read[i:]))
print(b) 

('b',('ett',('y_bought',('_butter_th',('$',1))('$',1))('er',('_butter_t',('$',1))('$',1)))('ought',('_b',('utter_th',('$',1))('etter_butter_t',('$',1)))('$',1))('utter_',('t',('h',('$',1))('$',1)('o_make_the_',('$',1)))('was_',('$',1)('bitter_',('$',1)))('better',('$',1)))('itt',('er_',('$',1)('b',('etty_bought',('$',1))('utter_better',('$',1))))('$',1)))
('e',('tt',('y_bought',('_butter_th',('$',1))('$',1))('er',('_butter_t',('$',1))('$',1)))('r',('_',('t',('h',('$',1)('e_butter_was_',('$',1)))('$',1)('o_make_the_',('$',1)))('was_',('$',1)('bitter_',('$',1)))('$',1)('b',('ett',('y_bought',('$',1))('er',('$',1)))('utter_',('t',('$',1))('better',('$',1)))))('$',1))('_',('b',('utter_was_',('$',1)('bitter_',('$',1)))('itt',('$',1)('er_butter_better',('$',1))))('the_',('$',1)('bitt',('$',1)('er_butter_better',('$',1))))('$',1)))
('t',('t',('y_bought',('_b',('utter_th',('$',1))('etter_butter_t',('$',1)))('$',1))('er',('_',('t',('h',('$',1)('e_butter_was_',('$',1)))('$',1)('o_make_the_',('$'

In [9]:
b = Branch()
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                                     'r_butter_to_make_the_',
                                                                                   'ke_the_bitter_butter_better']
for read in reads:
    for i in range(len(read)): b.add(Leaf(read[i:]))
print(b)

('b',('ett',('y_bought',('_butter_th',('$',1))('$',1))('er',('_butter_t',('$',1))('$',1)))('ought',('_b',('utter_th',('$',1))('etter_butter_t',('$',1)))('$',1))('utter_',('t',('h',('$',1))('$',1)('o_make_the_',('$',1)))('was_',('$',1)('bitter_',('$',1)))('better',('$',1)))('itter_',('$',1)('b',('etty_bought',('$',1))('utter_better',('$',1)))))
('e',('tt',('y_bought',('_butter_th',('$',1))('$',1))('er',('_butter_t',('$',1))('$',1)))('r',('_',('t',('h',('$',1)('e_butter_was_',('$',1)))('$',1)('o_make_the_',('$',1)))('was_',('$',1)('bitter_',('$',1)))('$',1)('b',('ett',('y_bought',('$',1))('er',('$',1)))('utter_',('t',('$',1))('better',('$',1)))))('$',1))('_',('b',('utter_was_',('$',1)('bitter_',('$',1)))('itter_butter_better',('$',1)))('the_',('$',1)('bitter_butter_better',('$',1)))('$',1)))
('t',('t',('y_bought',('_b',('utter_th',('$',1))('etter_butter_t',('$',1)))('$',1))('er',('_',('t',('h',('$',1)('e_butter_was_',('$',1)))('$',1)('o_make_the_',('$',1)))('was_',('$',1)('bitter_',('$',

In [8]:
print(b.b['h'])
print(b.b['t'].right['h'])
# print(b.b['_'].right['t'].right['h'])
# print(b.b['r'].right['_'].right['t'].right['h'])

('h',('t_',('rett',('ub_thguob_ytteb',('$',1))('$',1))('ek',('am_ot_rettub_r',('$',1))('$',1)))('guob_ytt',('eb',('$',1)('_rettib_sa',('$',1)))('$',1))('$',1))
('hguob_ytt',('eb',('$',1)('_rettib_sa',('$',1)))('$',1))


In [167]:
print('1',b.b['t'])
print('2',b.b['t'].right['t'])
print('3',b.b['t'].right['t'].right['e'])
print('4',b.b['t'].right['t'].right['e'].right['_'])
print('5',b.b['t'].right['t'].right['e'].right['_'].right['t'])
print('6',b.b['t'].right['t'].right['e'].right['_'].right['t'].right['h'])

1 ('t',('t',('y_bought',('_b',('utter_th',('$',1))('etter_butter_t',('$',1)))('$',1))('er',('_',('t',('h',('$',1)('e_butter_was_',('$',1)))('$',1)('o_make_the_',('$',1)))('was_',('$',1)('bitter_',('$',1)))('$',1)('b',('ett',('y_bought',('$',1))('er',('$',1)))('utter_',('t',('$',1))('better',('$',1)))))('$',1)))('y_bought',('_b',('utter_th',('$',1))('etter_butter_t',('$',1)))('$',1))('_b',('utter_th',('$',1))('etter_butter_t',('$',1)))('er',('_',('t',('h',('$',1)('e_butter_was_',('$',1)))('$',1)('o_make_the_',('$',1)))('was_',('$',1)('bitter_',('$',1)))('$',1)('b',('ett',('y_bought',('$',1))('er',('$',1)))('utter_',('t',('$',1))('better',('$',1)))))('$',1))('h',('$',1)('e_',('b',('utter_was_',('$',1))('itter_butter_better',('$',1)))('$',1)))('$',2)('o_make_the_',('$',1)))
2 ('t',('y_bought',('_b',('utter_th',('$',1))('etter_butter_t',('$',1)))('$',1))('er',('_',('t',('h',('$',1)('e_butter_was_',('$',1)))('$',1)('o_make_the_',('$',1)))('was_',('$',1)('bitter_',('$',1)))('$',1)('b',('ett'